## Diabetes Readmission - Exploratory Data Analysis

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
df = pd.read_csv('./data/diabetic_data_clean.csv', header=0)

### 1. Chi-square test of association between categorical variables and readmission.

#### Methods
The cleaned data set has 30 categorical variables. From previous categorical varaiable bar plot, it is hard to tell readmission is associated with which categorical variables. In order to test associations between categorical variables and readmission, We create 2-way contingency for each categorical variable vs readmission and complete Chi-square test on the table. The Chi-square test is based on a test statistic that measures the divergence of the observed data from the values that would be *expected* under the null hypothesis of no association. This requires calculation of the expected values based on the data. The expected value for each cell in a two-way table is equal to (row total x column total)/n, where n is the total number of observations included in the table.

Hemoglobin A1c (HbA1c) test can tell average level of blood sugar over the past 2 to 3 months. The rationale is that the blood suger, glucose, binds to the hemoglobin in red blood cells and the A1c test measures how much glucose is bound. Red blood cells live for about 3 months, so the test shows the average level of glucose in blood for the past 3 months. HbA1c test resulst is a indicator of diabetes. For people without diabetes, the normal range for the hemoglobin A1c level is between 4% and 5.6%. Hemoglobin A1c levels between 5.7% and 6.4% mean you have a higher chance of getting diabetes. Levels of 6.5% or higher mean you have diabetes. In this data set, the HbA1c results have 4 values: **>8** if the result was greater than 8%, **>7** if the result was greater than 7% but less than 8%, **normal** if the result was less than 7%, and **none** if not measured. Because HbA1c results were used in many studies to predict diabetes readmissions, the Chi-square test was carried out on this variable with readmission at different levels: H1bA1c vs Readmmission, level-combined H1bA1c VS Readimission, and H1bA1c + medication change vs Readmission.

#### Results
1. In primary study (no categorical level combination), 7 categorical variables have significant association with Readmission (*p*-value < 0.01). They are `repaglinide`, `age`, `diabetesMed`, `change`, `insulin`, `discharge_disposition_id` and `glipizide`. Except `age` and `discharge_disposition_id`, other 5 variables are all medication related, including medication types and medication changes.
 
 
2. `A1cresult` is not signifcant at primary study. After combining >7 category to norm category for `A1Cresult`, the association test generate a *p*-value of 0.0499. When adding one more variable `change` and create 3-way table, the *p*-value of association bwteeen **>8** A1Cresult/change of medications and readmission is 0.0999. But the *p*-value of association bwteeen **normal** A1Cresult/change of medications and readmission is 0.762.

In [3]:
def ch2_test(data, var):
    tbl_2way = pd.pivot_table(df, values='encounter_id', index=var, columns='LABEL', aggfunc='count').fillna(0)
    obs = np.array(tbl_2way)
    return round(stats.chi2_contingency(obs)[1],4)

In [4]:
vars_cat = list(df.columns.values[(df.dtypes == 'object').values]) + ['age', 'admission_type_id', 'discharge_disposition_id','admission_source_id']
p_value = []
for var in vars_cat:
    p = ch2_test(df, var)
    if p is np.nan:
        pass
    else:
        p_value.append(p)

In [5]:
df_cat_test = pd.DataFrame.from_dict({'cat_variables':vars_cat, 'p-value':p_value}).sort_values(by='p-value',ascending=1)
df_cat_test

,cat_variables,p-value
8,repaglinide,0.0000
28,diabetesMed,0.0000
29,age,0.0000
22,insulin,0.0000
31,discharge_disposition_id,0.0000
3,diag_2,0.0000
2,diag_1,0.0000
4,diag_3,0.0000
27,change,0.0001
7,metformin,0.0029


In [6]:
# combine >7 category to norm category for A1Cresult to create A1Cresult_1
df = df.assign(A1Cresult_1 = [A1Cresult if A1Cresult in ['None', "Norm"] else 'High' for A1Cresult in df['A1Cresult']])
print('p-value of association bwteeen >7 and > 8 A1Cresult and readmission is {:.4}.'.format(ch2_test(df, 'A1Cresult_1')))

p-value of association bwteeen >7 and > 8 A1Cresult and readmission is 0.033.


### 2. Permutation test on difference of mean of a numerical variable between two readmission categories.

#### Method
There are 8 numerical varialbes in the cleaned data set. Previous scatterplot shows readmission prefers low or high values of some numerical variables. Here we run statistical tests to test whether the difference of mean of a numerical variable between two Readmission categories is significant. From previous histgrams, the 8 numerical variables are not normally distributed. This means two-sample t-test can not be used here. Permutation test is a non-parametric methods and it can treat the situation that the distributions of the data are not normal. So permutation test was used. Each numerical variable was tested with readmission and generate *p*-value.

#### Results
Except two variables `num_procedures` and `number_outpatient` have *p*-values as 0.6626 and 0.0234, other 6 variables have *p*-value close to 0. The differences of mean of those 6 numerical variables between two Readmission categories are significant. Next we will test whether there are strong correlations between pairs of those numerical independent variables.

In [7]:
# concatnate two data and do permutation, then seperate them to two new data
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates
    
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

def permutation_test(data, var, var_label):
    # generate two data frames
    data_1 = data[var][data[var_label] == 0]
    data_2 = data[var][data[var_label] == 1]
    
    # Compute difference of mean impact force from experiment: empirical_diff_means
    empirical_diff_means = np.mean(data_1) - np.mean(data_2)

    # Draw 10,000 permutation replicates: perm_replicates
    perm_replicates = draw_perm_reps(data_1, data_2,
                                 diff_of_means, size=10000)

    # Compute p-value: p
    p = np.sum(abs(perm_replicates) >= abs(empirical_diff_means)) / len(perm_replicates)
    
    # Compute 95% confidence interval
    conf_int = np.percentile(perm_replicates, [2.5, 97.5])
    
    return p, conf_int

In [8]:
vars_num = ['num_procedures', 'number_diagnoses', 'num_medications', 'num_lab_procedures', 
            'time_in_hospital', 'number_outpatient', 'number_inpatient', 'number_emergency', 'num_change']

p_value = []
conf_int = []
for var in vars_num:
    p, ci = permutation_test(data=df, var=var, var_label='LABEL')
    p_value.append(p)
    conf_int.append(ci)

df_perm_test = pd.DataFrame.from_dict({'num_variables':vars_num, 
                                       'p-value':p_value, 
                                       'conf_int':conf_int}).sort_values(by='p-value',ascending=1)
df_perm_test

,num_variables,p-value,conf_int
1,number_diagnoses,0.0000,"[-0.05102672856175161, 0.0504798521830061]"
2,num_medications,0.0000,"[-0.21669630601978795, 0.2147110374290824]"
3,num_lab_procedures,0.0000,"[-0.5112449118076693, 0.5213395313856779]"
4,time_in_hospital,0.0000,"[-0.07647998076093288, 0.07542986766397952]"
6,number_inpatient,0.0000,"[-0.015865747836715405, 0.01528627177115835]"
7,number_emergency,0.0000,"[-0.013956710277114051, 0.012820025677968444]"
8,num_change,0.0167,"[-0.0184359057442558, 0.019016522323637264]"
5,number_outpatient,0.0239,"[-0.028413103171240683, 0.02671984613758184]"
0,num_procedures,0.9695,"[-0.04513774795821024, 0.04639318605818299]"


### 3. Check correlations of numeric indpendent variables.

#### Method
In order to investigate whether there are strong correlations between numerical variables, pair-wise Pearson's correlation coefficients and *p*-values were calcuated for thsoe 8 numerical variables. 

#### Results
For 8 numerical variables, there are 28 paris of correlation coefficients. Among those 28 pairs of correlation coefficients, 3 pairs of numerical variables have corrrelation coeffieients greater than 0.3. They are `num_medications` and `time_in_hospital` (*r*=0.47), `num_procedres` and `num_medications` (*r*=0.4), and `num_lab_precedures` and `time_in_hospital` (*r*=0.33).

In [9]:
num_variable1 = []
num_variable2 = []
corr_coeff =[]
p_value = []

n=0
for i in vars_num[:-1]:
    for j in vars_num[(n+1):]:
        r, p = stats.pearsonr(df[i], df[j])
        num_variable1.append(i)
        num_variable2.append(j)
        corr_coeff.append(round(r,2))
        p_value.append(p)
    n += 1
    
df_corr = pd.DataFrame.from_dict({'num_variable1':num_variable1, 
                                  'num_variable2':num_variable2,
                                  'corr_coeff':corr_coeff,
                                  'p-value':p_value}).sort_values(by='corr_coeff',ascending=0)
df_corr

,num_variable1,num_variable2,corr_coeff,p-value
16,num_medications,time_in_hospital,0.47,0.000000e+00
1,num_procedures,num_medications,0.40,0.000000e+00
21,num_lab_procedures,time_in_hospital,0.33,0.000000e+00
8,number_diagnoses,num_medications,0.26,0.000000e+00
15,num_medications,num_lab_procedures,0.26,0.000000e+00
20,num_medications,num_change,0.25,0.000000e+00
10,number_diagnoses,time_in_hospital,0.23,0.000000e+00
3,num_procedures,time_in_hospital,0.19,0.000000e+00
9,number_diagnoses,num_lab_procedures,0.15,0.000000e+00
33,number_inpatient,number_emergency,0.15,0.000000e+00
